In [1]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 92s - loss: 0.3225 - acc: 0.9007 - val_loss: 0.0765 - val_acc: 0.9764
Epoch 2/12
60000/60000 [==============================] - 97s - loss: 0.1103 - acc: 0.9676 - val_loss: 0.0502 - val_acc: 0.9834
Epoch 3/12
60000/60000 [==============================] - 92s - loss: 0.0832 - acc: 0.9757 - val_loss: 0.0419 - val_acc: 0.9863
Epoch 4/12
60000/60000 [==============================] - 96s - loss: 0.0727 - acc: 0.9787 - val_loss: 0.0399 - val_acc: 0.9865
Epoch 5/12
60000/60000 [==============================] - 95s - loss: 0.0631 - acc: 0.9813 - val_loss: 0.0347 - val_acc: 0.9883
Epoch 6/12
60000/60000 [==============================] - 94s - loss: 0.0549 - acc: 0.9841 - val_loss: 0.0311 - val_acc: 0.9892
Epoch 7/12
60000/60000 [==============================] - 95s - loss: 0.0509 - acc: 0.9845 - val_loss: 0.0306

In [13]:
from keras.layers import Input, Dense
from keras.models import Model

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(784,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

from keras.datasets import mnist
import numpy as np
(x_train, _), (x_test,_) = mnist.load_data()

# normalization
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# test size
print x_train.shape
print x_test.shape

autoencoder.fit(x_train, x_train, 
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))



(60000, 784)
(10000, 784)
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 2s - loss: 0.3792 - val_loss: 0.2735
Epoch 2/50
60000/60000 [==============================] - 2s - loss: 0.2676 - val_loss: 0.2587
Epoch 3/50
60000/60000 [==============================] - 2s - loss: 0.2498 - val_loss: 0.2385
Epoch 4/50
60000/60000 [==============================] - 2s - loss: 0.2307 - val_loss: 0.2207
Epoch 5/50
60000/60000 [==============================] - 2s - loss: 0.2148 - val_loss: 0.2063
Epoch 6/50
60000/60000 [==============================] - 2s - loss: 0.2020 - val_loss: 0.1951
Epoch 7/50
60000/60000 [==============================] - 2s - loss: 0.1921 - val_loss: 0.1865
Epoch 8/50
60000/60000 [==============================] - 2s - loss: 0.1844 - val_loss: 0.1796
Epoch 9/50
60000/60000 [==============================] - 2s - loss: 0.1781 - val_loss: 0.1739
Epoch 10/50
60000/60000 [==============================] - 2s - loss: